In [2]:
%load_ext autoreload
%autoreload 2

from DQN import DQN
import numpy as np
import pytorch_amd_hack_setup
import torch

ZLUDA device successfully loaded!


In [8]:
torch.autograd.set_detect_anomaly(True)

myModel = DQN(8, 4, 64).train()


inputData = [1]*8
print(inputData)

y = myModel(torch.tensor([inputData] * 8).float())
print(y)
print(y.cpu().gather(1, torch.tensor([1,2,3,0]*2).unsqueeze(1)).squeeze(-1))
print(y.cpu().max(1)[0])
print(y.cpu().gather(1, y.cpu().argmax(1).unsqueeze(1)).squeeze(-1))


# myModel.compile(
#      loss = 'mse',
#      metrics = ['mse'],
#      optimizer = tf.keras.optimizers.Adam(learning_rate=0.001))
# print(y)

[1, 1, 1, 1, 1, 1, 1, 1]
tensor([[-0.0093,  0.0422,  0.1062, -0.3558],
        [-0.0093,  0.0422,  0.1062, -0.3558],
        [-0.0093,  0.0422,  0.1062, -0.3558],
        [-0.0093,  0.0422,  0.1062, -0.3558],
        [-0.0093,  0.0422,  0.1062, -0.3558],
        [-0.0093,  0.0422,  0.1062, -0.3558],
        [-0.0093,  0.0422,  0.1062, -0.3558],
        [-0.0093,  0.0422,  0.1062, -0.3558]], grad_fn=<AddmmBackward0>)
tensor([ 0.0422,  0.1062, -0.3558, -0.0093,  0.0422,  0.1062, -0.3558, -0.0093],
       grad_fn=<SqueezeBackward1>)
tensor([0.1062, 0.1062, 0.1062, 0.1062, 0.1062, 0.1062, 0.1062, 0.1062],
       grad_fn=<MaxBackward0>)
tensor([0.1062, 0.1062, 0.1062, 0.1062, 0.1062, 0.1062, 0.1062, 0.1062],
       grad_fn=<SqueezeBackward1>)


In [25]:
from lunar import LunarLanderEnv

lunar = LunarLanderEnv(render_mode=None)
print(lunar.env.observation_space)
print(lunar.env.action_space.sample())
print(lunar.state)

Box([ -2.5        -2.5       -10.        -10.         -6.2831855 -10.
  -0.         -0.       ], [ 2.5        2.5       10.        10.         6.2831855 10.
  1.         1.       ], (8,), float32)
1
[-0.00583735  1.4208131  -0.59127843  0.43966836  0.00677083  0.13393345
  0.          0.        ]


c:\Users\miniluz\AppData\Local\pypoetry\Cache\virtualenvs\aprendizaje-modulo-lunar-5laruUkD-py3.10\lib\site-packages\pygame\pkgdata.py:25: UserWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html. The pkg_resources package is slated for removal as early as 2025-11-30. Refrain from using this package or pin to Setuptools<81.
  from pkg_resources import resource_stream, resource_exists


In [4]:
lunarState = lunar.state
lunarTensor = tf.convert_to_tensor([lunarState], dtype=np.float32)
result = myModel(lunarTensor)
print(np.argmax(result[0]))
print(result[0][1])

3
tf.Tensor(-0.02254521, shape=(), dtype=float32)


In [5]:
lunarState = lunar.state
lunarTensor = tf.convert_to_tensor([lunarState] * 100, dtype=np.float32)
actions = tf.convert_to_tensor([0, 1, 0, 3] * 25, dtype=np.int32)
# print(tf.one_hot(actions, 4))
print(myModel(lunarTensor) * tf.one_hot(actions, 4))
result = tf.reduce_sum(myModel(lunarTensor) * tf.one_hot(actions, 4), 1)
print(result)

tf.Tensor(
[[-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.         -0.          0.31878754]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.         -0.          0.31878754]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.         -0.          0.31878754]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.         -0.          0.31878754]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.02254522 -0.          0.        ]
 [-0.00458358 -0.         -0.          0.        ]
 [-0.         -0.   

In [6]:
lunarState = lunar.state
states = tf.convert_to_tensor([lunarState] * 100, dtype=np.float32)
next_states =  tf.convert_to_tensor([lunarState] * 100, dtype=np.float32)
rewards =  tf.convert_to_tensor([1] * 100, dtype=np.float32)
dones =  tf.convert_to_tensor([0] * 100, dtype=np.float32)
optimizer = keras.optimizers.Adam(learning_rate=0.01)

with tf.GradientTape() as tape:
    # Get Q-values for the actions that were actually taken
    q_values = tf.reduce_sum(myModel(states) * tf.one_hot(actions, 4), axis=1)
    
    # Get maximum Q-value for the next states from target network
    next_q_values = tf.reduce_max(myModel(next_states), axis=1)
    
    # Compute the expected Q-values
    expected_q_values = rewards + 0.9 * next_q_values * (1 - dones)
    
    # Compute the loss between the current and expected Q values
    loss = tf.keras.losses.MeanSquaredError()(expected_q_values, q_values)

gradients = tape.gradient(loss, myModel.trainable_weights)
for gradient in gradients:
    print(gradient.shape)

optimizer.apply_gradients(zip(gradients, myModel.trainable_weights))

(8, 64)
(64,)
(64, 64)
(64,)
(64, 4)
(4,)


<Variable path=adam/iteration, shape=(), dtype=int64, value=1>

In [16]:
l = np.array([1, 2, 3, 4])
choices = np.random.choice(len(l), size=2, replace=False, p=l/l.sum())
print(l[choices])

[3 4]


In [15]:
from DQN import *
buffer = DQNAgentDoubleDPrioritizedReplayMemory()

buffer.sample(1, beta=0.4)
for i in range(100):
    buffer.push(
        state=[i]*8,
        action=1,
        reward=1,
        next_state=[i]*8,
        done=0,
    )

print(buffer.sample(10, beta=0.4))
buffer.update_priorities(np.array([1,2,3]), np.array([100000]*3))
print(buffer.sample(3, beta=0.4))


(array([42, 28, 36, 27, 15, 52, 35, 96, 83, 84]), array([[42., 42., 42., 42., 42., 42., 42., 42.],
       [28., 28., 28., 28., 28., 28., 28., 28.],
       [36., 36., 36., 36., 36., 36., 36., 36.],
       [27., 27., 27., 27., 27., 27., 27., 27.],
       [15., 15., 15., 15., 15., 15., 15., 15.],
       [52., 52., 52., 52., 52., 52., 52., 52.],
       [35., 35., 35., 35., 35., 35., 35., 35.],
       [96., 96., 96., 96., 96., 96., 96., 96.],
       [83., 83., 83., 83., 83., 83., 83., 83.],
       [84., 84., 84., 84., 84., 84., 84., 84.]], dtype=float32), array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1], dtype=int64), array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1.], dtype=float32), array([[42., 42., 42., 42., 42., 42., 42., 42.],
       [28., 28., 28., 28., 28., 28., 28., 28.],
       [36., 36., 36., 36., 36., 36., 36., 36.],
       [27., 27., 27., 27., 27., 27., 27., 27.],
       [15., 15., 15., 15., 15., 15., 15., 15.],
       [52., 52., 52., 52., 52., 52., 52., 52.],
       [35., 35., 35., 35., 35., 